In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install einops yacs timm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 114.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [ ]:
!git clone https://github.com/swz30/Restormer.git
%cd Restormer


Cloning into 'Restormer'...
remote: Enumerating objects: 309, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 309 (delta 67), reused 56 (delta 56), pack-reused 202 (from 1)
Receiving objects: 100% (309/309), 1.56 MiB | 28.51 MiB/s, done.
Resolving deltas: 100% (123/123), done.
/content/Restormer


In [ ]:
import zipfile

zip_path = "/content/drive/My Drive/Image_Sharpening/blur_patches.zip"
extract_dir = "/content/blur_patches"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("✅ blur patches unzipped!")


✅ blur patches unzipped!


In [ ]:
!pip install lmdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.8/297.8 kB 17.0 MB/s eta 0:00:00


In [ ]:
# 4. Import everything & setup paths
import os
import sys
import torch
from PIL import Image
from torchvision import transforms
import torch.nn.functional as F

# Add Restormer to Python path
restormer_root = '/content/Restormer'
sys.path.insert(0, restormer_root)

from basicsr.models.archs.restormer_arch import Restormer

# ========== CONFIG ==========

# Change these according to your Drive structure
drive_root = '/content/drive/MyDrive/Image_Sharpening'  # or any folder you choose

input_folder = "/content/blur_patches/blur_patches"
output_png_folder = os.path.join(drive_root, 'teacher_patches')
output_pt_folder = os.path.join(drive_root, 'teacher_pt')
pretrained_model_path = os.path.join(drive_root, 'single_image_defocus_deblurring.pth')

os.makedirs(output_png_folder, exist_ok=True)
os.makedirs(output_pt_folder, exist_ok=True)


In [ ]:
# 5. Preprocessing
to_tensor = transforms.ToTensor()
to_pil = transforms.ToPILImage()

In [ ]:
# 6. Load model and weights (on GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Restormer().to(device)

checkpoint = torch.load(pretrained_model_path, map_location=device)
model.load_state_dict(checkpoint['params'], strict=True)
model.eval()


Restormer(
  (patch_embed): OverlapPatchEmbed(
    (proj): Conv2d(3, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (encoder_level1): Sequential(
    (0): TransformerBlock(
      (norm1): LayerNorm(
        (body): WithBias_LayerNorm()
      )
      (attn): Attention(
        (qkv): Conv2d(48, 144, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (qkv_dwconv): Conv2d(144, 144, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=144, bias=False)
        (project_out): Conv2d(48, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (norm2): LayerNorm(
        (body): WithBias_LayerNorm()
      )
      (ffn): FeedForward(
        (project_in): Conv2d(48, 254, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (dwconv): Conv2d(254, 254, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=254, bias=False)
        (project_out): Conv2d(127, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
    )
    (1): TransformerBlock(
 

In [ ]:
# Inference loop with progress bar
from tqdm import tqdm # Import tqdm
import os # Ensure os is imported
print(f"files: {len(input_folder)}")
png_files = [f for f in os.listdir(input_folder) if f.lower().endswith('.png')]
print(f"Total .png files found: {len(png_files)}")

# Ensure output directories exist
os.makedirs(output_png_folder, exist_ok=True)
os.makedirs(output_pt_folder, exist_ok=True)

with torch.no_grad():
    for filename in tqdm(png_files, desc="Processing patches"):
        input_path = os.path.join(input_folder, filename)
        try:
            img = Image.open(input_path).convert("RGB")
            img_tensor = to_tensor(img).unsqueeze(0).to(device)  # (1, 3, H, W)

            # Pad if needed to multiple of 8
            h, w = img_tensor.shape[2], img_tensor.shape[3]
            pad_h = (8 - h % 8) % 8
            pad_w = (8 - w % 8) % 8
            img_tensor = F.pad(img_tensor, (0, pad_w, 0, pad_h), mode='reflect')

            output_tensor = model(img_tensor)

            # Unpad and clamp
            output_tensor = output_tensor[:, :, :h, :w].clamp(0, 1).cpu()

            # Save PNG
            output_image = to_pil(output_tensor.squeeze(0))
            output_png_path = os.path.join(output_png_folder, filename)
            output_image.save(output_png_path)

            # Save tensor
            output_pt_path = os.path.join(output_pt_folder, filename.replace('.png', '.pt'))
            torch.save(output_tensor.squeeze(0), output_pt_path)

        except Exception as e:
            print(f"Error processing {filename}: {e}")


print("🎯 All patches processed and saved.")

files: 34
Total .png files found: 4723


Processing patches: 100%|██████████| 4723/4723 [21:36<00:00,  3.64it/s]

🎯 All patches processed and saved.
